In [ ]:
"""
This block gets all the popular repo data from Github
"""
import requests
import json
target_stars = 5000
date_since = '2000-11-06'
json_content = {'data': []}
for i in xrange(1,11):
    url = 'https://api.github.com/search/repositories?q=created:%3E{}+stars:%3E{}&sort=stars&order=desc&per_page=100&page={}'.format(date_since, target_stars, i)
    res = requests.get(url)
    json_content['data'] += res.json()['items']

In [ ]:
"""
This block saves the data to a file in json format
"""
file_name = 'data_json.txt'
with open(file_name, 'w') as f:
    f.write(json.dumps(json_content, sort_keys=True, indent=4, separators=(',', ': ')))

In [ ]:
"""
Testing
"""
import pprint
with open(file_name, 'r') as f:
    data = json.load(f)
    print len(data['data'])
print json.dumps(data['data'][0], sort_keys=True, indent=4, separators=(',', ': '))